# Session 4

In this session, we will build and train a neural network for the EGFR activity prediction task using **Pytorch**. 

**Content:**
1. What is Pytorch?
2. Tensor
3. Dataset and DataLoader
4. Defining the Neural Network (MLP)
5. Training
6. Testing

In [ ]:
!pip install PyTDC
import torch
print(torch.__version__)

In [ ]:
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

## What is Pytorch?
From [Nvidia](https://www.nvidia.com/en-us/glossary/pytorch/):
> **PyTorch** is a fully featured framework for building deep learning models, which is a type of machine learning that’s commonly used in applications like image recognition and language processing. Written in Python, it’s relatively easy for most machine learning developers to learn and use. PyTorch is distinctive for its excellent support for GPUs and its use of reverse-mode auto-differentiation, which enables computation graphs to be modified on the fly. This makes it a popular choice for fast experimentation and prototyping.


`torch.Tensor` is PyTorch's base data structure ("data abstraction") to represent 1D (vector), 2D (matrix) or higher dimentional data.

In [ ]:
# we can create a vector of only zeros
m_0 = torch.zeros(3)
print(m_0)

# or a 3 * 3 matrix of random numbers
m_1 = torch.rand(3, 3, dtype=torch.int)
print(m_1)

## Dataset and DataLoader
[PyTorch](https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html):
> PyTorch provides two data primitives: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` that allow you to use pre-loaded datasets as well as your own data. `Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the Dataset to enable easy access to the samples.

As we use our own dataset, we need to create a new **Dataset** class (a class that inherits from the `Dataset` superclass). This class needs to implement the following functions:
* `__init__`: the constructor function
* `__len__`: number of samples in dataset
* `__getitem__`: get a sample from the dataset at given index 

In [ ]:
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem

fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)

def smilesToFp(smiles):
    fpgen.GetFingerprintAsNumPy(Chem.MolFromSmiles(smiles))

In [ ]:
class ActivityDataset(Dataset):
    def __init__(self, smiles, activity):
        self.fingerprints = torch.tensor(list(map(smilesToFp, smiles)), dtype=torch.float)
        self.labels = torch.tensor(activity, dtype=torch.float)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        fingerprint = self.fingerprints[idx]
        label = self.labels[idx]
        return fingerprint, label

Let's first load our EGFR dataset:

In [ ]:
egfr_activities_df = pd.read_csv(
    "https://github.com/volkamerlab/cic_summerschool_2025/raw/refs/heads/main/data/EGFR-activities-prepared.csv",
    index_col=0,
)
egfr_activities_df.head()

And create an `ActivityDataset` instance from the full dataset:

In [ ]:
dataset = ActivityDataset(egfr_activities_df['canonical_smiles'], egfr_activities_df['pIC50'])

We split the dataset into a training set (75%), validation set (10%), and test set (15%) using PyTorch `random_split` function: TODO @Andrea, what are good split numbers here?

In [ ]:
n_train = int(0.7 * len(dataset))
n_valid = int(0.1 * len(dataset))
n_test = len(dataset) - n_train - n_valid

train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [n_train, n_valid, n_test], generator=torch.Generator().manual_seed(42)
)

For each split, we also create a `DataLoader` instance:

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

## Defining the Neural Network (MLP)
Here, we will build a simple **feedforward neural network** (multi-layer perceptron):
- Input layer: 1024 (size of the fingerprint)
- Hidden layer: 512 neurons, with ReLU activation
- Output layer: 1 neuron (predicted pIC50 value)

To build such a neural network, we need to create a subclass of PyTorch's base class `nn.Module`. In this subclass, we need to define an `__init__` function, but we also overwrite the `forward` function that defines the forward pass.

In [ ]:
class ActivityNet(nn.Module):
    def __init__(self):
        super(ActivityNet, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training
To train our MLP, we first create an instance of NN that we train `ActivityNet`. As loss function - on which the gradient is determined -, we will use the MSE. Further, we will use the optimizer `optim.Adam` for the parameter updates.

In [ ]:
model = ActivityNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

We train the model in *mini-batches*, which are small groups of samples defined by the `batch_size` in the DataLoader. This means that in every **training step** we update the model on all `batches` seperatly. 

Let's define one **training step**:

In [ ]:
def train_step(loader):
    model.train()
    total_loss = 0
    
    # we iterate over all batches
    for fingerprints, labels in train_dataloader:
        
        # zero the gradients, 
        # otherwise the new gradients will be added to the from the last iteration gardients 
        optimizer.zero_grad()
        
        # forward step
        outputs = model(fingerprints)
        
        # empirical loss (sqeeze() to tranform a (N,1) tensor (matrix) to (N,) tensor (vector))
        loss = criterion(outputs.squeeze(), labels)
        
        # compute the gradients
        loss.backward()
        
        # updates parameters (weights & bias)
        optimizer.step()
        total_loss += loss.item() * len(labels)
        
    return total_loss / len(loader.dataset)

After each training step, we test the model on the validation data:

In [ ]:
def test_step(loader):
    model.eval()
    valid_loss = 0.0
    
    # do not change the gradients
    with torch.no_grad():
        for fingerprints, labels in valid_dataloader:
            outputs = model(fingerprints)
            valid_loss += criterion(outputs.squeeze(), labels).item()
    valid_loss /= len(valid_dataloader)
    return valid_loss

We are now, ready to train the NN in *epochs*:

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    train_loss = train_step(train_dataloader)
    valid_loss = test_step(valid_dataloader)

    print(f'Epoch: {epoch + 1:03d}, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')

## Testing

In [ ]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for fingerprints, labels in test_dataloader:
        outputs = model(fingerprints)
        test_loss += criterion(outputs.squeeze(), labels).item()
test_loss /= len(test_dataloader)
print(f'Test Loss: {test_loss:.4f}')

In [ ]:
with torch.no_grad():
    sns.regplot(x=model(test_dataloader.dataset.fingerprints).flatten(), y=test_dataloader.dataset.labels, scatter_kws={"s": 10})
plt.ylabel("True value")
plt.xlabel("Predicted value")
plt.show()

TODO place somewhere: Useful links:
* python classes: https://docs.python.org/3/tutorial/classes.html
* Dataloader & Dataset: https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html
* Step by step pythorch tut:
* Tensors tut: 

## Exercises

### Exercise 1

In [ ]:
# Create a MLP that has 4 layers: input layer -> hidden layer with 256 neurons and ReLU activation -> hidden layer with  512 neurons and sigmoid activation -> output layer

# TODO

In [ ]:
# Train the model using the Adams optimizer and the MSE loss function

# TODO

In [ ]:
# Test this model

# TODO

In [ ]:
# Challenge: can you improve the runtime by changeing the number of neurons in the hidden layers?

## Challenge